### DATASET & DATALOADER 살펴복
- Pytorch에서 배치크기만 데이터를 조절하는 메커니즘
- Dataset ; 사용 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoarder : 지정된 Dataset에서 지정된 batchsize만큼 피처와 타겟을 추출하여 전달함


1. 모듈 로딩 및 데이터 준비 <hr>

In [1]:
## 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

2 - 2. 사용자 정의 데이터셋 생성하기

In [304]:
## 데이터 준비하기
iris = pd.read_csv('../datas/iris.csv')

In [305]:
iris.info

<bound method DataFrame.info of      sepal.length  sepal.width  petal.length  petal.width    variety
0             5.1          3.5           1.4          0.2     Setosa
1             4.9          3.0           1.4          0.2     Setosa
2             4.7          3.2           1.3          0.2     Setosa
3             4.6          3.1           1.5          0.2     Setosa
4             5.0          3.6           1.4          0.2     Setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  Virginica
146           6.3          2.5           5.0          1.9  Virginica
147           6.5          3.0           5.2          2.0  Virginica
148           6.2          3.4           5.4          2.3  Virginica
149           5.9          3.0           5.1          1.8  Virginica

[150 rows x 5 columns]>

In [306]:
irisnp = np.loadtxt('../datas/iris.csv', delimiter=',',skiprows= 1, usecols=[0,1,2,3])

In [307]:
irisnp[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [308]:
#type check
if iris.__class__.__name__ == 'DataFrame':
    print('DF')
else:
    print('-----')

DF


In [309]:
print(isinstance(iris,pd.DataFrame), isinstance(irisnp,pd.DataFrame), isinstance(iris,np.ndarray), isinstance(irisnp,np.ndarray),sep='\n')

True
False
False
True


In [310]:
# [2 - 1]사용자 정의 dataset class 만들기
# - data tensorize
# - 
class DLdataset(Dataset):
    
    #초기화 함수 콜백함수 (callback funcation)
    def __init__(self, xdata, ydata):
        super().__init__()
        
        xdata = xdata.values if isinstance(xdata,pd.DataFrame) else xdata #xdata 타입체크, np.ndarray 변환
        ydata = ydata.values if isinstance(ydata,pd.DataFrame) else ydata #ydata 타입체크, np.ndarray 변환    
        
        #tensorize
        self.feature=torch.FloatTensor(xdata)
        self.label=torch.FloatTensor(ydata) #lable encoding
    
    #데이터셋 갯수 체크 함수 콜백함수(callback funcation)
    def __len__(self):
        return len(self.label) #return self.target.shape[0]    
    
    #특정 인덱스 데이터 + 라벨 반환 콜백함수(callback funcation)
    def __getitem__(self, index):
        return self.feature[index],self.label[index]
    

In [311]:
#feature/target 분리
feature = iris[iris.columns[:-1]]
target = iris[iris.columns[-1]]#.to_frame() #column 하나만 빼면 시리즈니까 DF로 바꿔주는데, 라벨인코더 먼저 쓰기


#object -> int 타입으로 전환
from sklearn.preprocessing import LabelEncoder
targetnp = LabelEncoder().fit_transform(target)
targetnp = targetnp.reshape(-1,1) #1차원 데이터를 2차원으로 늘려주기


print(f'feature ; {feature.shape}, {feature.ndim}D')
print(f'target : {target.shape},{target.ndim}D')



feature ; (150, 4), 2D
target : (150,),1D


In [337]:
#데이터셋 생성하기: DF, NP

mydata = DLdataset(feature, targetnp) #np파일로 입력

In [ ]:
#데이터셋 생성: np. np
mydata2 = DLdataset(irisnp,targetnp)
print(mydata2[0])

In [ ]:
mydata[0]

In [313]:
feature.iloc[0]

sepal.length    5.1
sepal.width     3.5
petal.length    1.4
petal.width     0.2
Name: 0, dtype: float64

In [314]:
target[0]

'Setosa'

2 - 3. 학습용 검증용, 테스트용 Dataset 분리하기 <hr>

In [315]:
###Pytorch
from torch.utils.data import random_split

#학습용, 검증용, 테스트데이터 비율
seed= torch.Generator().manual_seed(42)
train,valid,test = random_split(mydata2, [0.7,0.1,0.2],generator=seed) #train_Test_split()

print(f'subset attributes: \nindecies: {train.indices}\ndataset : {train.dataset}')

subset attributes: 
indecies: [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12]
dataset : <__main__.DLdataset object at 0x000001E27251A340>


3. DataLoader 생성하기: 학습용, 검증용, 테스트용 <hr>


In [316]:
#DataLoader 생성
#Drop_last : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 (버리기) : 기본 = False
batch = 10
trainDL = DataLoader(train,batch_size=batch)
validDL = DataLoader(valid,batch_size=batch)
testDL = DataLoader(test, batch_size=batch)

len(trainDL), len(validDL),len(testDL) 

(11, 2, 3)

In [317]:
#Epoch당 반복단위
print(f'batchsize = {batch}')
print(f'trainDL: {len(trainDL)}개, validDS: {len(validDL)}개, testDL: {len(testDL)}개')
print(f'trainDS:{len(train)}개, validDS:{len(valid)}개, testDS:{len(test)}개')

batchsize = 10
trainDL: 11개, validDS: 2개, testDL: 3개
trainDS:105개, validDS:15개, testDS:30개


4. 모델 클래스 정의하기: 입출력 피처 수, 층(layer) 수, 은닉층의 노드(퍼셉트론/유닛) 수 <hr>

- 구조 설계
    * 입력층 : 입력 4(입력 = 피처 개수)
    * 은닉층 : 알잘딱해서 아무렇게나...
    * 출력층 : 출력 ([분류] 출력 = 타겟 클래스 개수, [회귀] 1개)

In [318]:
#모델 클래스 정의
#클래스명 : c-model

class CModel(torch.nn.Module): #nn.Module 상속
    # 구조 설계
    # 구성 요소 정의 함수
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self. input_layer = nn.Linear(in_dim,100 )
        self.relu = nn.ReLU()
        self.hidden_layer = nn.Linear(  100,27  )
        self.output_layer = nn.Linear(  27, out_dim)    
    
    #순방향 학습
    def forward(self,x):
        y = self.input_layer(x)   #W1x1+W2x2+....Wnxn+b 100개 반환
        y = self.relu(y)          # relu() 결과 100개 반환
        y = self.hidden_layer(y)  #W1x1+W2x2+....Wnxn+b 27개 반환
        y = self.relu(y)          # relu() 결과 27개 반환
        y = self.output_layer(y)     #W1x1+W2x2+....Wnxn+b 3개 반환
        return y 

5. 학습을 위한 준비: 실행 디바이스, 모델, 최적화, 손실함수, 학습 횟수, 학습 함수, 평가 함수, 예측함수 
<hr>



In [319]:
# 실행 디바이스 설정하기
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

#학습 횟수
EPOCHS = 50


In [320]:
# 모델 인스턴스
in_dim, out = mydata2.feature.shape[1], len(np.unique(target))
model = CModel(in_dim,out).to(DEVICE)
print(f'in {in_dim}, out {out}')

in 4, out 3


In [321]:
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE) #다중분류 = crossentropy(이진분류 = binarycrossentropy)

#최적화 인스턴스
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

- 학습 및 검증 관련 함수 정의하기

In [322]:

## 학습 진행함수
def training():
    #학습모드 : 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    #배치 크기만큼 학습 진행 및 저장
    train_loss=[]
    for cnt, (feature, target) in enumerate(trainDL):
        #print(cnt, feature, target)
        #배치 크기만큼 학습 데이터 준비
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        target = target.squeeze()
        #학습
        pre_target = model(feature)
        #print(f'pre_target->{pre_target.shape},{pre_target.ndim}D')
        #print(f'target=> {target.shape},{target.ndim}D')
        
        #t손실 계산
        print(pre_target,target)
        loss = LOSS_FN(pre_target, target.long())
        train_loss.append(loss)
        
        #W,b update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #배치단위 학습진행 메시지 출력
        print(f'[train{cnt} batch loss] = {loss}')
    
    #에포크 단위 학습 진행 메시지 출력
    print(f'train loss : {loss}')
    
    return train_loss




In [323]:
training()

tensor([[-0.2159, -0.0708, -0.2037],
        [-0.3280, -0.1453, -0.3129],
        [-0.2273, -0.0931, -0.1995],
        [-0.2968, -0.1228, -0.3186],
        [-0.3632, -0.1660, -0.3482],
        [-0.2099, -0.0870, -0.1942],
        [-0.4157, -0.1512, -0.3362],
        [-0.2127, -0.0789, -0.2110],
        [-0.3196, -0.1301, -0.3154],
        [-0.2170, -0.0860, -0.2110]], grad_fn=<AddmmBackward0>) tensor([0., 1., 0., 1., 1., 0., 2., 0., 1., 0.])
[train0 batch loss] = 1.080426812171936
tensor([[-0.3589, -0.0484, -0.3502],
        [-0.2831, -0.0538, -0.3425],
        [-0.3043, -0.0419, -0.3525],
        [-0.2886, -0.0792, -0.3692],
        [-0.3824, -0.0907, -0.4048],
        [-0.3071, -0.0410, -0.3651],
        [-0.1321, -0.0467, -0.2436],
        [-0.1699, -0.0858, -0.2999],
        [-0.2466, -0.0501, -0.3232],
        [-0.3333, -0.0166, -0.3829]], grad_fn=<AddmmBackward0>) tensor([2., 1., 1., 1., 2., 2., 0., 0., 1., 2.])
[train1 batch loss] = 1.0835354328155518
tensor([[-0.3669, -0.0111, 

[tensor(1.0804, grad_fn=<NllLossBackward0>),
 tensor(1.0835, grad_fn=<NllLossBackward0>),
 tensor(1.0656, grad_fn=<NllLossBackward0>),
 tensor(1.0345, grad_fn=<NllLossBackward0>),
 tensor(1.0287, grad_fn=<NllLossBackward0>),
 tensor(0.9906, grad_fn=<NllLossBackward0>),
 tensor(0.9988, grad_fn=<NllLossBackward0>),
 tensor(0.9868, grad_fn=<NllLossBackward0>),
 tensor(1.0042, grad_fn=<NllLossBackward0>),
 tensor(0.9624, grad_fn=<NllLossBackward0>),
 tensor(0.9117, grad_fn=<NllLossBackward0>)]

In [324]:
import torchmetrics.functional as metrics

In [333]:
## 검증 / 평가 진행 함수
### ==> 검증 및 평가 진행함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader
def testing(dataloader):
    # 추론 모드 -> 정규화, 경사하강법, 드랍아웃 등의 기능 비활성화
    model.eval()

    with torch.no_grad(): # 경사하강법 끄기
        #model.eval()
        # 배치크기 만큼 학습 진행 및 저장
        val_loss=[]
        for cnt, (feature,target) in enumerate(validDL):
            # 배치크기 만큼 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()

            # 학습
            pre_target = model(feature)

            # 손실계산
            loss = LOSS_FN(pre_target,target.long())
            val_loss.append(loss)

            # 배치 단위 학습 진행 메세지 출력
    # 에퍼크 단위 학습 진행 메시지 출력
    acc = metrics.accuracy(pre_target, target, task='multiclass',num_classes=3)
    print(f'[Valid loss]==>{loss} [Valid loss]==>{acc}')

    return val_loss

In [334]:
testing(validDL)

[Valid loss]==>0.5425488352775574 [Valid loss]==>1.0


[tensor(0.4712), tensor(0.5425)]

In [335]:
## 예측 함수
def predict():
    pass

6. 학습 진행하기 <hr>

In [336]:
# 지정된 횟수만큼 처음부터 끝까지 학습 및 검증 진행하기
# 목표 : 최적의 (오차 최소화) W,b를 가진 모델 완성
# 조기 종료 기능 : 조건; val_loss가 지정된 횟수(ex: 5)이상 개선되지 않으면 학습 종료
for eps in range(EPOCHS):
    # 학습
    train_loss = training()
    # 검증
    valid_loss = testing(validDL)
    
    print(f'[{eps}/{EPOCHS}] TRAiN{sum(train_loss) / len(train_loss)},\nVALID:{sum(valid_loss)/len(valid_loss)}\n')

tensor([[ 1.6502, -0.6066, -1.3664],
        [-0.9729,  0.5812,  0.1871],
        [ 1.4945, -0.5069, -1.3087],
        [-0.6231,  0.4525, -0.0278],
        [-1.2473,  0.7129,  0.2910],
        [ 1.8580, -0.6878, -1.5291],
        [-2.0467,  0.8738,  1.0091],
        [ 1.8936, -0.7030, -1.5320],
        [-0.8340,  0.5591,  0.0515],
        [ 1.8745, -0.6933, -1.5299]], grad_fn=<AddmmBackward0>) tensor([0., 1., 0., 1., 1., 0., 2., 0., 1., 0.])
[train0 batch loss] = 0.37950602173805237
tensor([[-2.0294,  0.8401,  1.0519],
        [-1.1746,  0.6433,  0.3421],
        [-1.4158,  0.7374,  0.4968],
        [-1.0102,  0.6454,  0.1366],
        [-2.2217,  0.8982,  1.1603],
        [-1.6056,  0.7288,  0.7354],
        [ 1.2591, -0.3623, -1.1848],
        [ 1.6014, -0.5657, -1.3591],
        [-0.6759,  0.4841, -0.0059],
        [-2.0016,  0.7864,  1.0958]], grad_fn=<AddmmBackward0>) tensor([2., 1., 1., 1., 2., 2., 0., 0., 1., 2.])
[train1 batch loss] = 0.5449918508529663
tensor([[-1.7413,  0.7280